In [88]:
import re 
import sys 
import os 
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import time 
from math import log2

In [113]:
# 1. https://www.tensorflow.org/text/tutorials/text_generation
# 2. https://www.thepythoncode.com/article/text-generation-keras-python
# 3. https://www.kaggle.com/code/hommelette/neural-language-model-with-tensorflow

In [5]:
lang  = 'cwe' # 'sw'
path_file = os.path.join('data', lang+'-train.txt')
cwe = open(path_file, 'r').read().lower()
cwe_decoded =  open(path_file, 'rb').read().decode(encoding = 'utf-8')


In [6]:
unique_chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'

In [7]:
character_list = re.findall(r"[%s]"%unique_chars, cwe_decoded)

In [8]:
chars = tf.strings.unicode_split(cwe_decoded.split(), input_encoding='UTF-8')

Metal device set to: Apple M1


2022-10-25 14:51:30.277288: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-25 14:51:30.277633: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
chars[0]

<tf.Tensor: shape=(7,), dtype=string, numpy=array([b'c', b'h', b'i', b'k', b'a', b'l', b'e'], dtype=object)>

In [10]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(unique_chars), mask_token=None)

In [11]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[25, 30, 31, 33, 23, 34, 27], [44, 31, 36, 37, 29, 31, 34, 27],
 [28, 23, 36, 23], ..., [36, 47, 31, 36, 29, 31], [33, 43, 28, 37, 41, 23],
 [45, 37, 41, 27, 9]]>

In [12]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [13]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'c', b'h', b'i', b'k', b'a', b'l', b'e'],
 [b'v', b'i', b'n', b'o', b'g', b'i', b'l', b'e'],
 [b'f', b'a', b'n', b'a'], ..., [b'n', b'y', b'i', b'n', b'g', b'i'],
 [b'k', b'u', b'f', b'o', b's', b'a'], [b'w', b'o', b's', b'e', b'.']]>

In [14]:
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'chikale', b'vinogile', b'fana', ..., b'nyingi', b'kufosa',
       b'wose.'], dtype=object)

In [15]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis = -1)

In [16]:
all_ids = ids_from_chars(tf.strings.unicode_split(cwe_decoded, 'UTF-8'))
all_ids

<tf.Tensor: shape=(603432,), dtype=int64, numpy=array([25, 30, 31, ..., 27,  9,  0])>

In [17]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [18]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))


c
h
i
k
a
l
e
 
v
i


In [19]:
seq_length = 100

In [20]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'c' b'h' b'i' b'k' b'a' b'l' b'e' b' ' b'v' b'i' b'n' b'o' b'g' b'i'
 b'l' b'e' b' ' b'f' b'a' b'n' b'a' b' ' b'v' b'i' b'y' b'a' b' ' b'w'
 b'a' b'n' b'h' b'u' b' ' b'w' b'o' b'c' b'h' b'i' b'k' b'a' b'l' b'a'
 b' ' b'h' b'a' b'b' b'u' b'n' b'g' b"'" b'h' b'u' b'k' b'e' b' ' b'i'
 b'm' b'i' b's' b'i' b',' b' ' b's' b'i' b' ' b'k' b'w' b'a' b' ' b'u'
 b'm' b'e' b'l' b'o' b' ' b'n' b'a' b' ' b'u' b'n' b'g' b"'" b'w' b'a'
 b'j' b'i' b' ' b'n' b'a' b' ' b'u' b'n' b'y' b'o' b'l' b'o' b'd' b'o'
 b' ' b'n' b'a'], shape=(101,), dtype=string)


In [21]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b"chikale vinogile fana viya wanhu wochikala habung'huke imisi, si kwa umelo na ung'waji na unyolodo na"
b' uwashelati na ndwagi hebu migongo. one muhapula chinhu chochose kwa zina jangu nizamtendelani. yesu '
b'kamulongela petili, "bweleza zele jako muna iyala. "ufalume wa kuulanga ulinga vino. niye mulala mula'
b'ngulizi, nokwandikila weye gayo mulondwa wangu yonikulonda muna ikweli. nomulongelani ukweli, munhu y'
b'oyose yohauhokela ufalume wa mulungu fana mwana mdoododo, hezakwingila muna ufalume uwo ng\'o!" iyo ni'


In [22]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [23]:
split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [24]:
dataset = sequences.map(split_input_target)


In [25]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"chikale vinogile fana viya wanhu wochikala habung'huke imisi, si kwa umelo na ung'waji na unyolodo n"
Target: b"hikale vinogile fana viya wanhu wochikala habung'huke imisi, si kwa umelo na ung'waji na unyolodo na"


2022-10-25 14:51:34.866788: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [26]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [27]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [28]:
class Model(tf.keras.Model):
    def __init__(self, char_size, embedding_size, run_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

In [29]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [30]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [31]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    

(64, 100, 49) # (batch_size, sequence_length, vocab_size)


In [32]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  12544     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  50225     
                                                                 
Total params: 4,001,073
Trainable params: 4,001,073
Non-trainable params: 0
_________________________________________________________________


In [33]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'msenga wa kuulanga wekaidi kamsondelela msenga wa kuulanga wa mwanduso na kalonga, "kugwa! yesu kamu'

Next Char Predictions:
 b'h1)1fe4ef4z41r8b mb4?x((tdpe!q-[UNK]3.2w6()nqve6!,2"-?jrnj"qvtcm3zrtpckus9xb:v";knrdp)n3;v ebxemm58j5\'a,'


In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 49)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.8933127, shape=(), dtype=float32)


In [37]:
tf.exp(example_batch_mean_loss).numpy()

49.073177

In [38]:
model.compile(optimizer='adam', loss=loss)


In [39]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [40]:
EPOCHS = 20


In [41]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20


2022-10-25 14:51:42.473937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 14:51:42.822756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 14:51:43.148265: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


93/93 [==============================] - 19s 185ms/step - loss: 2.6758
Epoch 2/20
93/93 [==============================] - 17s 185ms/step - loss: 1.9501
Epoch 3/20
93/93 [==============================] - 17s 184ms/step - loss: 1.8419
Epoch 4/20
93/93 [==============================] - 18s 186ms/step - loss: 1.7543
Epoch 5/20
93/93 [==============================] - 18s 187ms/step - loss: 1.6489
Epoch 6/20
93/93 [==============================] - 17s 185ms/step - loss: 1.5247
Epoch 7/20
93/93 [==============================] - 17s 184ms/step - loss: 1.4165
Epoch 8/20
93/93 [==============================] - 17s 180ms/step - loss: 1.3207
Epoch 9/20
93/93 [==============================] - 17s 180ms/step - loss: 1.2526
Epoch 10/20
93/93 [==============================] - 17s 182ms/step - loss: 1.1875
Epoch 11/20
93/93 [==============================] - 17s 183ms/step - loss: 1.1365
Epoch 12/20
93/93 [==============================] - 17s 182ms/step - loss: 1.0889
Epoch 13/20
93/93 [=====

In [79]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

  @tf.function
  def generate_one_step_probablity(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

#     Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
#     predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_logits, states

In [80]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [81]:
start = time.time()
states = None
next_char = tf.constant(['R'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

2022-10-25 16:44:28.017361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 16:44:28.060532: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 16:44:28.246981: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 16:44:28.287694: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


R. "habule munhu yanogile kyafulile ulopozi wa mizoyo yawo wana ndugu yake, muhe wa mulungu kwa ubazi wake, cheye chidale sulumi wa wayahudi waitendelela munhu yoyowahituma yang'hali hawanisang'hanile." vilagusiligwa muna zimbuli zose, fana tata yeli hamwe na pnulotu na nyang'hiti si chibumbila cha mkala. sang'hano wowotunhiza wano, mbali na munhu yoyose yoiyahaza weze kwa munhu yakalile yolondeka kupeta mbuli zihile. maabaho yesu kamulongela yakalizi wa dibululu mboli kumwangu dizifa na kubalila mbeyo muna iisi. mbali niye si mulungu waggibila mumavi. na mweye mwamunailumba na mnyeni yalongiligwe na muhe wa mulungu na watumigwa waja wakamhawo wagane na ugima!" viyahulike vino ndagusigwa bule, kwaviya niye nombunza chilisito. luti na daganaka ndolo wa kunitogola. ivo noilumba na wanhu wose wahulumuka kumulamsa, 'tata wa mulungu wangu bila kudumba na kuitendela ubazi wa mndewa. fana viya vimito momilemwa na kutogola vino kwaviya uhasanyi, na vizawvunga va kiuvisanya na motosa siku ya mn

In [105]:
loss = 0 

start = time.time()
states = None
next_char = tf.constant(['r'])
result = []

total_loss_list = []
for n in range(100):
    next_probablity, states = one_step_model.generate_one_step_probablity(next_char, states=states)
    total_loss_list.append(-np.log2(tf.nn.softmax(next_probablity.numpy()[0], axis=None))) 
    result.append(next_probablity)

# result = tf.strings.join(result)
end = time.time()
# print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
# print('\nRun time:', end - start)



/var/folders/h4/6v442zlj5pn5g0738xkh07940000gn/T/ipykernel_7756/1364750804.py:11: RuntimeWarning: divide by zero encountered in log2
  total_loss_list.append(-np.log2(tf.nn.softmax(next_probablity.numpy()[0], axis=None)))


In [112]:
np.sum(total_loss_list, axis =  0) # len(total_loss_list)

array([       inf,  583.6647 ,  568.996  ,  457.28238,  522.8706 ,
        847.0175 ,  717.7082 ,  552.27136,  577.0855 , 1065.0303 ,
        765.56494,  751.5368 ,  761.3358 ,  802.7334 ,  729.5246 ,
        673.6226 ,  804.05884,  756.1036 ,  779.15857,  802.0268 ,
        738.14154,  779.24725,  850.688  ,  492.58273,  552.7007 ,
       1115.8677 ,  667.4169 ,  731.8954 ,  527.58417,  722.6052 ,
        744.3494 ,  451.85962,  572.219  ,  426.95538,  419.58127,
        556.4994 ,  435.3779 ,  824.32697,  534.9528 ,  754.5881 ,
        661.45624,  601.9297 ,  414.7642 ,  684.0325 ,  625.3372 ,
        660.2923 ,  763.1272 ,  647.49866,  629.3153 ], dtype=float32)

In [87]:
loss_from_scratch -= log2(from_scratch_model(lang, c, history))

tf.nn.softmax(
    result[3].numpy(), axis=None, name=None
)

<tf.Tensor: shape=(1, 49), dtype=float32, numpy=
array([[0.        , 0.01102065, 0.0045095 , 0.00373487, 0.0011907 ,
        0.00677215, 0.00610704, 0.00579459, 0.01328425, 0.00617429,
        0.00732338, 0.0060174 , 0.0053658 , 0.00600239, 0.00693956,
        0.00694221, 0.00585509, 0.00660033, 0.00604411, 0.00697283,
        0.00720852, 0.00630329, 0.00389412, 0.06216983, 0.04352461,
        0.0074307 , 0.01922443, 0.0281563 , 0.03188287, 0.0179441 ,
        0.01944627, 0.04947171, 0.01321515, 0.02109216, 0.0960482 ,
        0.02188122, 0.05470426, 0.03344189, 0.02975821, 0.00666699,
        0.00667353, 0.03088338, 0.0883512 , 0.06332189, 0.01017965,
        0.02915852, 0.00754721, 0.02856939, 0.03919932]], dtype=float32)>

In [45]:
start = time.time()
states = None
next_char = tf.constant(['R', 'R', 'R', 'R', 'R'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

2022-10-25 14:57:37.688264: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 14:57:37.757019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 14:57:37.997745: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 14:57:38.038844: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


tf.Tensor(
[b'Rbulizi wa bululu. yano viyahalike kupotigwa na ndugu zako wezadilemela bule. viwakomeleze kumkumbu na mwanage kuyotela. ivo mulungu kotaza kwa mulungu, komuhitila masodika, na iviya waja yolonga kuwa nolonga na kumwingila munhu ino. wopeta matumbala mote yadenganike wakala wahulika hbulu yake, na inikona kukonhela mweye kwa mbuli yoyose, muladi yaloneeehimawa na welasigwa? yesu kamuhonyanhu haja hachihondigwe muulongo na kufosela muulumita. baho niya ndima yotangigwa "mbuli ya wfulungu, nikumbasafunyo ng\'o, vino mulungu kakala kolonga nawo wamfungta. mbali chiumono na yoyandikigwe mwanza umwenga na makanduku hamwe na lukali. wanhu wose wezamuwona hanhu hondu yohanda. ipangi yoyaninogeligwe, yoisafykkani mweye wenyewo kumwetu wiza na kugong\'onda ukala mogololeni mayoja na kusondelela malungu. yesu kawahonya wetu na koilanga yano yoyayatangigwe udaho wa mndewa, mbali wakala wokolola weye kwawichiwona, na wasang\'hani wa weye, fana viyilonda mulungu tati yetu yelu mgelonk

In [46]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

2022-10-25 14:57:44.926878: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets
2022-10-25 14:57:52.268971: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 14:57:52.274489: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [47]:
states = None
next_char = tf.constant(['chond'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

2022-10-25 14:57:52.310306: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 14:57:52.401095: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 14:57:52.524702: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-25 14:57:52.587841: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


chonde cholonda kulonga, sekeyamulolele mulungu yoyowatenda wano wawotambwanw  ndugu zangu." saabi moto m
